In [9]:
# examples with classes

class Calculator:
    def __init__(self, name):
        self.name = name  # Store data in this instance
    
    def add(self, a, b):
        print(f"{self.name} is adding {a} + {b}")  # Access self.name
        return a + b

# Usage:
calc = Calculator("MyCalc")
summy = calc.add(2, 3)  # Python automatically passes calc as self
print(f"The computed sum is {summy}\n")

class Duck:
    def __init__(self, name):
        self.name = name  # Store the duck's name
    
    def introduce(self):
        print(f"Quack! My name is {self.name}")
    
    def quack(self):
        print("Quack!")

# Usage examples:
donald = Duck("Donald")
donald.introduce()
donald.quack()

MyCalc is adding 2 + 3
The computed sum is 5

Quack! My name is Donald
Quack!
